In [1]:
# Depth Camera
import pyrealsense2.pyrealsense2 as rs

# Widgets
from IPython.display import display
import ipywidgets

# Basic libraries
from threading import Thread
import numpy as np
import PIL.Image
import copy
import time
import json
import cv2

# Pytorch modules
from trt_pose.parse_objects import ParseObjects
from trt_pose.draw_objects import DrawObjects
import torchvision.transforms as transforms
from torch2trt import TRTModule
import trt_pose.models
import torch2trt
import trt_pose.coco
import trt_pose
import torch

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-4rbv3pjz because the default path (/home/jetbot/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


# Functions to prepare cameras

In [ ]:
def FindMyCameras():
    ctx = rs.context()
    devices = [x for x in list(ctx.query_devices())]
    idxs = [str(x).find('S/N') for x in devices]
    devices = [str(x)[y+5:y+17] for x,y in zip(devices,idxs)]
    return devices

In [ ]:
def PrepareCamera(pipeline, serialNo, useRGB):
    config = rs.config()
    pipeline_wrapper = rs.pipeline_wrapper(pipeline)
    pipeline_profile = config.resolve(pipeline_wrapper)
    config.enable_device(serialNo)
    config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
    
    if useRGB:
        config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)
    return config

In [ ]:
def InitializeCameras(number_of_cameras):
    cameras = FindMyCameras()
    frontCam = rs.pipeline()
    frontCamConfig = PrepareCamera(frontCam, cameras[0], useRGB=True)
    profile_front = frontCam.start(frontCamConfig)
    depth_sensor = profile_front.get_device().first_depth_sensor()
    front_depth_scale = depth_sensor.get_depth_scale()
    
    if number_of_cameras == 2:
        backCam = rs.pipeline()
        backCamConfig = PrepareCamera(backCam, cameras[1], useRGB=False)
        profile_back = backCam.start(backCamConfig)
        depth_sensor = profile_back.get_device().first_depth_sensor()
        back_depth_scale = depth_sensor.get_depth_scale()
        print('Cameras ready to start streaming! :)')
        return frontCam, backCam, front_depth_scale, back_depth_scale
    
    print('Camera ready to start streaming! :)')
    return frontCam, frontCam, front_depth_scale, front_depth_scale

# Functions to calculate angles

In [ ]:
''' Returns angle to the robot'''
def GetAngle(keypoints):
    try:
        angle = calculate_angle(640, keypoints)

    except:
        angle = [0]
    return angle

In [ ]:
def calculate_angle(WIDTH, keypoints, FOV=69):

    keys = keypoints.values()
    
    # if at least 2 points are detected
    if len(keys) > 1:  
        object_location = int(sum([x for x,y in keys])/len(keys))
        angle = int(FOV*(WIDTH/2 - object_location)/WIDTH)
        return [angle]
    
    
    else:
        return [0]

In [ ]:
''' Updates Angle if there was a difference > threshold'''
def UpdateAngle(angle, noiseThreshold):
    
    global global_angle
    
    if global_angle:
        if abs(global_angle-angle[0])>2:
            if angle[0] != 0:
                global_angle = angle[0]

            else:
                global_angle = 0
                
    else:
        global_angle = angle[0]   
        
    global_angle*=(-1)

# Functions to calculate distance

In [ ]:
''' Calculates the distance to the human '''
def CalculateDistanceToHuman(depth_image, keypoints):  
    global global_depth
    global last_correct_distance
    # If Human is detected, calculate Depth
    
    combination_shoulders = ['left_shoulder', 'right_shoulder']
    combination_hips = ['left_hip', 'right_hip']
    combination_elbows = ['left_elbow', 'right_elbow']
    
    try:
        if len(keypoints.keys()) >= 2:
            
            # If both shoulders are available, use chest for distance
            if combination_shoulders[0] in keypoints.keys() and combination_shoulders[1] in keypoints.keys():
                center_coordinate_x = int((keypoints['left_shoulder'][0]+keypoints['right_shoulder'][0])/2)
                center_coordinate_y = int((keypoints['left_shoulder'][1]+keypoints['right_shoulder'][1])/2)
                depth = float(depth_image[center_coordinate_y, center_coordinate_x])
                depth = depth#/frontScale  # use depth calibration
                                          
            
            # If both shoulders are hips, use lower belly for distance
            elif combination_hips[0] in keypoints.keys() and combination_hips[1] in keypoints.keys():
                center_coordinate_x = int((keypoints['left_hip'][0]+keypoints['right_hip'][0])/2)
                center_coordinate_y = int((keypoints['left_hip'][1]+keypoints['right_hip'][1])/2)
                depth = float(depth_image[center_coordinate_y, center_coordinate_x])
                depth = depth#/frontScale  # use depth calibration
                
            
            # If both shoulders are hips, use lower belly for distance
            elif combination_elbows[0] in keypoints.keys() and combination_elbows[1] in keypoints.keys():
                center_coordinate_x = int((keypoints['left_elbow'][0]+keypoints['right_elbow'][0])/2)
                center_coordinate_y = int((keypoints['left_elbow'][1]+keypoints['right_elbow'][1])/2)
                depth = float(depth_image[center_coordinate_y, center_coordinate_x])
                depth = depth#/frontScale  # use depth calibration
                
            else:
                depth = np.mean([depth_image[keypoints[key][1],keypoints[key][0]] for key in keypoints.keys()])
                depth = depth#/frontScale  # use depth calibration
            
            
            if depth != 0:
                last_correct_distance = depth   
            else:
                depth = last_correct_distance
            
            #depth = float(np.mean([depth_image[keypoints[x][0], keypoints[x][1]] for x in keypoints.keys()]))
            global_depth = depth
        
        else:
            global_depth = last_correct_distance

    # Otherwise, set Depth = 1
    except:
        global_depth = 3000

In [ ]:
''' Calculate distance reading at each cross '''
def CalculateDistanceToRays(angles, x_coordinates, depth):
    
    global ray_distances
    
    for idx,angle in enumerate(angles):
        
        x = x_coordinates[idx]
        temp_depth = depth[240, x]
        
        if int(abs(ray_distances[angle] - temp_depth)/5) > 10 and temp_depth >= 300:
            
            ray_distances[angle] = int(temp_depth)

        elif int(abs(ray_distances[angle] - temp_depth)/5) > 10 and temp_depth < 300 and temp_depth > 0:

            ray_distances[angle]=0


In [ ]:
''' Calculates distances for backward facing rays '''
def MeasureBackwardDistances(rays_coordinates):
    frame = GetFrames(backwardCamera, getColor=False)
    distances = [frame[240,x] for x in rays_coordinates]
    distances = [round(float(x/1000), 3) for x in distances]
    distances = [x if x <=1 else 1 for x in distances]
    return distances

# Robot Actions

In [ ]:
''' Takes actions '''
def step(action):
   
    # Extract speeds
    try:
        action = action.cpu().detach().numpy()[0]
        
    except:
        action = action
        
    speed_move, speed_turn = float(abs(action[0])), float(abs(action[1]))
    
    speed_move = (((speed_move - 0) * (MAX_LIN_VEL - 0)) / (1 - 0)) + 0
    speed_turn = (((speed_turn - 0) * (MAX_ANG_VEL - 0)) / (1 - 0)) + 0
    
    turtlebotMove(speed_move, time_move=TIME_MOVE)
    time.sleep(TIME_MOVE + DELAY_BETWEEN_ACTIONS)  # Must include TIME_MOVE otherwise robot takes both actions simultaneoulsy
    turtlebotRotate(speed_turn, time_rotate=TIME_ROTATE)

In [ ]:
''' Obtain actions from the Neural Network '''
def sample_action(observation):
    
    global agent
    
    observation = torch.Tensor(observation).cuda()
    hidden,_ = agent.network_body(vis_inputs=[0],vec_inputs=[observation])
    distribution = agent.distribution(hidden)[0]
    action = distribution.sample()
    action = torch.clamp(action, -3, 3)/3
    
    return action    

In [ ]:
''' Use Neural Network to move robot '''
def MoveRobot(observation):
    # Move if person is more than 0.8 distance units away
    if observation[3] < 0.6 and observation[0]==1:
        action = [0, 0]

    else:
        action = sample_action(observation)
        step(action)

        # Extract action for printing info
        action = action.cpu().detach().numpy()[0]
        speed_move, speed_turn = float(abs(action[0])), float(abs(action[1]))
        action = [speed_move, speed_turn]
        
    return action

# Functions related to frame processing & drawing

In [ ]:
def bgr8_to_jpeg(value, quality=75):
    return bytes(cv2.imencode('.jpg', value)[1])

In [ ]:
def preprocess(image):
    global device
    image = cv2.resize(image, (224,224))
    device = torch.device('cuda')
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device)
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

In [ ]:
def get_keypoints(image, human_pose, topology, object_counts, objects, normalized_peaks):
    height = image.shape[0]
    width = image.shape[1]
    keypoints = {}
    K = topology.shape[0]
    count = int(object_counts[0])

    for i in range(count):
        obj = objects[0][i]
        C = obj.shape[0]
        for j in range(C):
            k = int(obj[j])
            if k >= 0:
                peak = normalized_peaks[0][j][k]
                x = round(float(peak[1]) * width)
                y = round(float(peak[0]) * height)
                keypoints[human_pose["keypoints"][j]] = (x, y)

    return keypoints

In [ ]:
''' Draw skeleton and rays on the frames '''
def DrawOnFrames(color_image, counts, objects, peaks):   
    global image_to_nn
    draw_objects(color_image, counts, objects, peaks)    
    color_image = draw_rays(color_image, rays_coordinates, rays_in_angles, ray_distances)
    image_to_nn = copy.deepcopy(color_image)

In [ ]:
''' Reads One Camera and returns Frames as arrays'''
def GetFrames(camera, getColor=False):
    align_to = rs.stream.color
    align = rs.align(align_to)
    frames = camera.wait_for_frames()
    aligned_frames = align.process(frames)
    depth_frame = aligned_frames.get_depth_frame()
    depth_image = np.asanyarray(depth_frame.get_data())

    if getColor:
        color_frame = aligned_frames.get_color_frame()
        color_image = np.asanyarray(color_frame.get_data())
        color_image = cv2.flip(color_image, 1)
        return color_image, depth_image
    else:
        return depth_image

In [ ]:
def CalculateKeyPoints(color_image):
    data = preprocess(color_image)
    cmap, paf = model_trt(data)
    cmap, paf = cmap.detach().cpu(), paf.detach().cpu()
    counts, objects, peaks = parse_objects(cmap, paf)
    keypoints = get_keypoints(color_image, human_pose, topology, counts, objects, peaks)
    return keypoints, counts, objects, peaks

In [ ]:
''' Draw the crosses on the left frame (just for reference) '''
def draw_rays(image, ray_positions, angles, distances):
    
    # Mirror image so text written is correct
    image = cv2.flip(image, 1)
    
    red_color = (0,0,255)
    yellow_color = (0,255,255)
    green_color = (0,255,0)
    
    # Draw a '+' at every ray position & write the angle that cross belongs to
    for idx,ray in enumerate(ray_positions):
        
        # Assign cross color based on distance
        distance_to_object = distances[angles[idx]]/10
        cross_color = lambda x: yellow_color if x< 80 and x >=50 else red_color if x < 50 else green_color
        color = cross_color(distance_to_object)

        # Draw the cross
        cv2.line(image, (ray+3,240), (ray-3,240), color, 2)    # Horizontal line
        cv2.line(image, (ray, 240-3), (ray, 240+3), color, 2)  # Vertical line
        
        # Write what angle the cross belongs to        
        cv2.putText(image, str(angles[idx]), org=(ray-15, 240+22), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.5, color=color, thickness=1)
        
    # Return to normal
    image = cv2.flip(image, 1)
    return image

# Functions to handle the UI

### General UI Functions (both left and right)

In [ ]:
''' Create the UI '''
def CreateUI():
    
    global image_w, rays_widget, distance_widget, angle_widget
    global RL_image_w, RL_rays_widget, RL_distance_widget, RL_angle_widget, RL_stage_actions_widget
    
    # Real Time Widgets
    image_w = ipywidgets.Image(format='jpeg', width=480, height=480)
    rays_widget = ipywidgets.HTML()
    distance_widget = ipywidgets.HTML()
    angle_widget = ipywidgets.HTML()

    text_widget = ipywidgets.VBox([distance_widget, rays_widget, angle_widget], layout=ipywidgets.Layout(width='100%'))
    final_real_time = ipywidgets.HBox([image_w, text_widget], layout=ipywidgets.Layout(width='100%', align_self='center'))
    final_real_time.add_class("box_style")

    # RL widgets
    RL_image_w = ipywidgets.Image(format='jpeg', width=480, height=480)
    RL_rays_widget = ipywidgets.HTML()
    RL_distance_widget = ipywidgets.HTML()
    RL_angle_widget = ipywidgets.HTML()

    RL_text_widget = ipywidgets.VBox([RL_distance_widget, RL_rays_widget, RL_angle_widget], layout=ipywidgets.Layout(width='100%'))
    RL_final_real_time = ipywidgets.HBox([RL_text_widget, RL_image_w], layout=ipywidgets.Layout(width='100%', align_self='center'))
    RL_final_real_time.add_class("RL_box_style")

    RL_stage_actions_widget = ipywidgets.HTML()
    RL_stage_actions_widget.add_class("RL_stage_actions_style")

    results = ipywidgets.HBox([RL_final_real_time, final_real_time], layout=ipywidgets.Layout(width='100%', align_self='center'))
    results = ipywidgets.VBox([results, RL_stage_actions_widget], layout=ipywidgets.Layout(width='100%'))
    
    display(results)

In [ ]:
%%html
<style>
.RL_box_style{
    width:auto;
    border : 1px solid red;
    height: auto;
    background-color:Thistle;
}
.box_style{
    width:auto;
    border : 1px solid green;
    height: auto;
    background-color:PaleGreen;
}
.RL_stage_actions_style{
    width:auto;
    border : 1px solid blue;
    height: auto;
    background-color:LightCyan;
}
</style>

### For the Widgets of the right side

In [ ]:
def UpdateWidgets(rays_in_angles): 
    rays_widget.value = f"<pref><font color='blue'>Distance to each ray (Raw Readings)<br> {rays_in_angles[0]}: {ray_distances[rays_in_angles[0]]/10}cm<br> {rays_in_angles[1]}: {ray_distances[rays_in_angles[1]]/10}cm<br> {rays_in_angles[2]}: {ray_distances[rays_in_angles[2]]/10}cm<br> {rays_in_angles[3]}: {ray_distances[rays_in_angles[3]]/10}cm<br> {rays_in_angles[4]}: {ray_distances[rays_in_angles[4]]/10}cm<br> {rays_in_angles[5]}: {ray_distances[rays_in_angles[5]]/10}cm<br> {rays_in_angles[6]}: {ray_distances[rays_in_angles[6]]/10}cm<br> {rays_in_angles[7]}: {ray_distances[rays_in_angles[7]]/10}cm<br> {rays_in_angles[8]}: {ray_distances[rays_in_angles[8]]/10}cm<br> {rays_in_angles[9]}: {ray_distances[rays_in_angles[9]]/10}cm<br> {rays_in_angles[10]}: {ray_distances[rays_in_angles[10]]/10}cm<br> {rays_in_angles[11]}: {ray_distances[rays_in_angles[11]]/10}cm<br> {rays_in_angles[12]}: {ray_distances[rays_in_angles[12]]/10}cm<br>"             
    distance_widget.value = f"<pref><font color='red'>Distance to person: {int(global_depth/10)}cm"
    angle_widget.value = f"<pref><font color='purple'>Angle to person: {int(global_angle)}"
    image_w.value = bgr8_to_jpeg(color_image[:, ::-1, :])

### For the Widgets of the left side

In [ ]:
''' Normalizes the widget values '''
def NormalizeWidgetValues():
    angles_to_NN = [round(float(ray_distances[x]/1000), 3) for x in ray_distances]
    angles_to_NN = [x if x <=1 else 1 for x in angles_to_NN ]
    distance_to_person_NN = float(global_depth/1000)
    if distance_to_person_NN >= 3:
        distance_to_person_NN = 1
    else:
        distance_to_person_NN=round(distance_to_person_NN/3,2)
        
    return distance_to_person_NN, angles_to_NN

In [ ]:
''' Updates widgets on the left '''
def UpdateDelayedWidgets(distance_to_person_NN, angles_to_NN):
    RL_rays_widget.value = f"<pref><font color='blue'>Distance to each ray (normalized) <br> {rays_in_angles[0]}: {angles_to_NN[0]}<br> {rays_in_angles[1]}: {angles_to_NN[1]}<br> {rays_in_angles[2]}: {angles_to_NN[2]}<br> {rays_in_angles[3]}: {angles_to_NN[3]}<br> {rays_in_angles[4]}: {angles_to_NN[4]}<br> {rays_in_angles[5]}: {angles_to_NN[5]}<br> {rays_in_angles[6]}: {angles_to_NN[6]}<br> {rays_in_angles[7]}: {angles_to_NN[7]}<br> {rays_in_angles[8]}: {angles_to_NN[8]}<br> {rays_in_angles[9]}: {angles_to_NN[9]}<br> {rays_in_angles[10]}: {angles_to_NN[10]}<br> {rays_in_angles[11]}: {angles_to_NN[11]}<br> {rays_in_angles[12]}: {angles_to_NN[12]}<br>"             
    RL_distance_widget.value = f"<pref><font color='red'>Distance to person: {distance_to_person_NN}m"
    RL_angle_widget.value = f"<pref><font color='purple'>Angle to person: {int(global_angle)}"
    RL_image_w.value = bgr8_to_jpeg(image_to_nn[:, ::-1, :])

In [ ]:
''' Calculates the stage '''
def DetermineStage():
    # Define Stage
    try:
        # If everything is visible, stage = 1
        if 'left_ankle' in keypoints.keys() and 'right_ankle' in keypoints.keys() and 'left_knee' in keypoints.keys() and 'right_knee' in keypoints.keys():
            stage = [1,0,0]

        # Else, stage = 2
        else:
            if len(keypoints.keys()) >= 2:
                stage = [0,1,0]
            else:
                stage = [0,0,1]
    except:
        stage = [0,0,1]

    return stage